In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/YoloV5DataSet.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
except:
    print("Using Local Machine")
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt


In [ ]:
# Include all packages
import gc
import cv2
import shutil
import random
import numpy as np
import pandas as pd
from time import time
from copy import deepcopy

from datetime import datetime
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import lr_scheduler


from yolov5.models.yolo import Model
from yolov5.utils.loss import ComputeLoss
import yolov5.val as validate

from yolov5.utils.dataloaders import create_dataloader
from yolov5.utils.general import colorstr, intersect_dicts, check_amp, labels_to_class_weights, strip_optimizer, TQDM_BAR_FORMAT
from yolov5.utils.autobatch import check_train_batch_size
from yolov5.utils.torch_utils import EarlyStopping, ModelEMA, smart_optimizer, de_parallel

import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:
def CreateDataLoaders(trainPath, valPath,  imageSize, batchSize, gridSize, hyperParameter, numWorkers, options):
    trainLoader, dataset = create_dataloader(trainPath,
                                              imageSize,
                                              batchSize,
                                              gridSize,
                                              hyp=hyperParameter,
                                              workers=numWorkers,
                                              rect=options.rect,
                                              quad=options.quad,
                                              seed=options.seed,
                                              prefix=colorstr('train: '),
                                              image_weights=options.image_weights,
                                              cache=None if options.cache == 'val' else options.cache,
                                              augment=True,
                                              rank=-1,
                                              shuffle=True)
    valLoader = create_dataloader(valPath,
                                    imageSize,
                                    batchSize * 2,
                                    gridSize,
                                    hyp=hyperParameter,
                                    cache=None,
                                    rect=True,
                                    rank=-1,
                                    workers=numWorkers * 2,
                                    pad=0.5,
                                    prefix=colorstr('val: '))[0]
    
    labels = np.concatenate(dataset.labels, 0)

    return trainLoader, valLoader, labels


In [ ]:
def CreateYolov5Model(numClasses: int, version: str, device, weightsFilePath, configFilePath, hyperParameters ):
    if(weightsFilePath != None):
        ckpt = torch.load(weightsFilePath, map_location='cpu')  # load checkpoint to CPU to avoid CUDA memory leak
        model = Model(configFilePath, ch=3, nc=numClasses, anchors=hyperParameters.get('anchors')).to(device)  # create
        exclude = ['anchor'] 
        csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
        csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
        model.load_state_dict(csd, strict=False)  # load
    else:
        model = Model(configFilePath, ch=3, nc=numClasses, anchors=hyperParameters.get('anchors')).to(device)  # create
    amp = check_amp(model)  # check AMP

    return model, amp


In [ ]:
def AddModelAttributes(model, hyperParameters, numClasses, imageSize, options, dataset, device, classNames):
    nl = de_parallel(model).model[-1].nl
    hyperParameters['box'] *= 3 / nl  
    hyperParameters['cls'] *= numClasses / 80 * 3 / nl  
    hyperParameters['obj'] *= (imageSize / 640) ** 2 * 3 / nl  
    hyperParameters['label_smoothing'] = options.label_smoothing
    model.nc = numClasses 
    model.hyp = hyperParameters
    model.class_weights = labels_to_class_weights(dataset.labels, numClasses).to(device) * numClasses  # attach class weights
    model.names = classNames
    return model, hyperParameters

In [ ]:
def GetConfig(model, imageSize, amp, hyperParameters, options, epochs, nominalBatchSize = 64):
    gridSize = max(int(model.stride.max()), 32)  # grid size (max stride)
    batchSize = check_train_batch_size(model, imageSize, amp)
    accumulate = max(round(nominalBatchSize / batchSize), 1)
    hyperParameters['weight_decay'] *= batchSize * accumulate / nominalBatchSize 
    optimizer = smart_optimizer(model, options.optimizer, hyperParameters['lr0'], hyperParameters['momentum'], hyperParameters['weight_decay'])

    # Scheduler
    lf = lambda x: (1 - x / epochs) * (1.0 - hyperParameters['lrf']) + hyperParameters['lrf'] 
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf) 

    # EMA
    ema = ModelEMA(model)


    return gridSize, batchSize,accumulate, optimizer, scheduler, ema


In [ ]:
def TrainModel(model, trainLoader,numEpochs, ema, batchSize, device,optimizer,  hyperParameters, lf, numClasses, options, scheduler, amp, patience=100, nominalBatchSize= 64 ):
    numBatchs = len(trainLoader) 
    numWarmup = max(round(hyperParameters['warmup_epochs'] * numBatchs), 100) 
    lastOptStep = -1
    mAPc = np.zeros(numClasses)
    results = (0, 0, 0, 0, 0, 0, 0)
    scheduler.last_epoch = - 1
    scaler = torch.cuda.amp.GradScaler(enabled=amp)
    earlyStopper, earlyStop = EarlyStopping(patience=patience), False
    computeLoss = ComputeLoss(model)  # init loss class
    for epoch in range(numEpochs):
        model.train()
        meamLoss = torch.zeros(3, device=device)  # mean losses
        pbar = tqdm(pbar, total=numBatchs, bar_format=TQDM_BAR_FORMAT)
        optimizer.zero_grad()
        for i, (imgs, targets, paths, _) in pbar:  
            numIntergrations = i + numBatchs * epoch  
            imgs = imgs.to(device, non_blocking=True).float() / 255 
            
            # Warmup
            if numIntergrations <= numWarmup:
                xi = [0, numWarmup]  # x interp
                # compute_loss.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
                accumulate = max(1, np.interp(numIntergrations, xi, [1, nominalBatchSize / batchSize]).round())
                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x['lr'] = np.interp(numIntergrations, xi, [hyperParameters['warmup_bias_lr'] if j == 0 else 0.0, x['initial_lr'] * lf(epoch)])
                    if 'momentum' in x:
                        x['momentum'] = np.interp(numIntergrations, xi, [hyperParameters['warmup_momentum'], hyperParameters['momentum']])
            
            # Multi-scale
            # if opt.multi_scale:
            #     sz = random.randrange(int(imgsz * 0.5), int(imgsz * 1.5) + gs) // gs * gs  # size
            #     sf = sz / max(imgs.shape[2:])  # scale factor
            #     if sf != 1:
            #         ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to gs-multiple)
            #         imgs = nn.functional.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)
            
            # Forward
            with torch.cuda.amp.autocast(amp):
                pred = model(imgs)
                loss, lossItems = computeLoss(pred, targets.to(device))
                # if options.quad:
                #     loss *= 4.
            
             # Backward
            scaler.scale(loss).backward()

            # Optimize - https://pytorch.org/docs/master/notes/amp_examples.html
            if (numIntergrations - lastOptStep >= accumulate):
                scaler.unscale_(optimizer)  # unscale gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)  # clip gradients
                scaler.step(optimizer)  # optimizer.step
                scaler.update()
                optimizer.zero_grad()
                if( ema):
                    ema.update(model)
                lastOptStep = numIntergrations
        # Scheduler
        lr = [x['lr'] for x in optimizer.param_groups]  # for loggers
        scheduler.step()
        # if RANK in {-1, 0}:
        #     # mAP
        #     callbacks.run('on_train_epoch_end', epoch=epoch)
        #     ema.update_attr(model, include=['yaml', 'nc', 'hyp', 'names', 'stride', 'class_weights'])
        #     final_epoch = (epoch + 1 == epochs) or stopper.possible_stop
        #     if not noval or final_epoch:  # Calculate mAP
        #         results, maps, _ = validate.run(data_dict,
        #                                         batch_size=batch_size // WORLD_SIZE * 2,
        #                                         imgsz=imgsz,
        #                                         half=amp,
        #                                         model=ema.ema,
        #                                         single_cls=single_cls,
        #                                         dataloader=val_loader,
        #                                         save_dir=save_dir,
        #                                         plots=False,
        #                                         callbacks=callbacks,
        #                                         compute_loss=compute_loss)

        #     # Update best mAP
        #     fi = fitness(np.array(results).reshape(1, -1))  # weighted combination of [P, R, mAP@.5, mAP@.5-.95]
        #     stop = stopper(epoch=epoch, fitness=fi)  # early stop check
        #     if fi > best_fitness:
        #         best_fitness = fi
        #     log_vals = list(mloss) + list(results) + lr
        #     callbacks.run('on_fit_epoch_end', log_vals, epoch, best_fitness, fi)

        #     # Save model
        #     if (not nosave) or (final_epoch and not evolve):  # if save
        #         ckpt = {
        #             'epoch': epoch,
        #             'best_fitness': best_fitness,
        #             'model': deepcopy(de_parallel(model)).half(),
        #             'ema': deepcopy(ema.ema).half(),
        #             'updates': ema.updates,
        #             'optimizer': optimizer.state_dict(),
        #             'opt': vars(opt),
        #             'git': GIT_INFO,  # {remote, branch, commit} if a git repo
        #             'date': datetime.now().isoformat()}

        #         # Save last, best and delete
        #         torch.save(ckpt, last)
        #         if best_fitness == fi:
        #             torch.save(ckpt, best)
        #         if opt.save_period > 0 and epoch % opt.save_period == 0:
        #             torch.save(ckpt, w / f'epoch{epoch}.pt')
        #         del ckpt
        #         callbacks.run('on_model_save', last, epoch, final_epoch, best_fitness, fi)

        # # EarlyStopping
        # if RANK != -1:  # if DDP training
        #     broadcast_list = [stop if RANK == 0 else None]
        #     dist.broadcast_object_list(broadcast_list, 0)  # broadcast 'stop' to all ranks
        #     if RANK != 0:
        #         stop = broadcast_list[0]
        # if stop:
        #     break  # must break all DDP ranks
    #     last = 'last.pt'
    #     best = 'best.pt'
    #     for f in last, best:
    #         if f.exists():
    #             strip_optimizer(f)  # strip optimizers
    #             if f is best:
    #                 results, _, _ = validate.run(
    #                     data_dict,
    #                     batch_size=batch_size // WORLD_SIZE * 2,
    #                     imgsz=imgsz,
    #                     model=attempt_load(f, device).half(),
    #                     iou_thres=0.65 if is_coco else 0.60,  # best pycocotools at iou 0.65
    #                     single_cls=single_cls,
    #                     dataloader=val_loader,
    #                     save_dir=save_dir,
    #                     save_json=is_coco,
    #                     verbose=True,
    #                     plots=plots,
    #                     callbacks=callbacks,
    #                     compute_loss=compute_loss)  # val best model with plots

    # torch.cuda.empty_cache()
    # return results


In [ ]:
batchSize = 32
inputShape = 640
epochs = 100
numAnchors = 3
yolo5Version = 's'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
numClasses = 10

In [ ]:
print("Using {} device".format(device))


In [ ]:
print("Downloading Weights of yolo5 Verion ", yolo5Version)
weightsURL = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(
    yolo5Version)
!wget {weightsURL}


In [ ]:
yolov5Model = CreateYolov5Model(numClasses, yolo5Version, device)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.01)

yolov5Model = yolov5Model.to(device)
scheduler = ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=10, verbose=True)
